## COVID and the US Housing Crisis

The cost of housing in the United States has gone up dramatically since the start of the Covid-19 pandemic, both for rentals and single family homes. Meanwhile, the availability of single family housing is lower than ever. The reasons for this lack of availability are many: the cost of building materials has skyrocketed; builders are not able to keep up with demand; and homes go under contract almost as soon as listing. The dream of home ownership is further out of reach for the average family, and even more so for individuals. 

This project will attempt to quantify the factors leading to the state of the housing market today. The project begins with an examination of how much housing costs have gone up during the pandemic, both at the state and metro level. It will look at housing inventory, including how much inventory has changed over time, and how long homes take to sell once on the market. All analyses use data available from Zillow, a real estate search service (https://www.zillow.com/research/data/).

Zillow uses a methodology called Zillow Home Value Index (ZHVI), to calculate the typical home value for a given market, taking into account both location and seasonal adjustments. Data is published within a lower-tier (5th to 35th percentile), mid-tier (35th-65th percentile), and top-tier (65th to 95th percentile) range of home values. For the purposes of this project, the mid-tier data is used to represent the value of the average home for the middle-class.

In [1]:
#Importing libraries
import numpy as np
import pandas as pd

In [2]:
#Read in 4 files: home price by state, home price by metro, housing inventory, 
#and pending status (time for a home to go under contract)
home_state = pd.read_csv('State_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
home_metro = pd.read_csv('Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv')
inventory_metro = pd.read_csv('Metro_invt_fs_uc_sfrcondo_sm_month.csv') 
pending_metro = pd.read_csv('Metro_med_doz_pending_uc_sfrcondo_sm_month.csv')

In [3]:
#Information about the mid-tier home value by state
home_state.info()
home_state.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Columns: 275 entries, RegionID to 2022-06-30
dtypes: float64(270), int64(2), object(3)
memory usage: 109.7+ KB


,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
0,9,0,California,State,CA,201873.0,202731.0,203810.0,206082.0,208517.0,...,716941.0,721951.0,727440.0,734241.0,744446.0,756574.0,771569.0,786055.0,796986.0,799311.0
1,54,1,Texas,State,TX,114777.0,114902.0,114942.0,115150.0,115199.0,...,269194.0,273492.0,277149.0,281437.0,286128.0,291601.0,297048.0,303127.0,308619.0,313339.0
2,43,2,New York,State,NY,150441.0,151211.0,151854.0,153212.0,154439.0,...,370157.0,373228.0,376126.0,378994.0,382306.0,386106.0,390880.0,395849.0,400971.0,405105.0
3,14,3,Florida,State,FL,116126.0,116469.0,116832.0,117535.0,118205.0,...,319849.0,327258.0,334598.0,341431.0,349013.0,356969.0,366710.0,377432.0,388200.0,397280.0
4,21,4,Illinois,State,IL,148565.0,148890.0,149334.0,150205.0,151102.0,...,243117.0,245371.0,247651.0,250176.0,252547.0,254932.0,257629.0,260502.0,263716.0,266415.0


This dataset contains home values from the year 2000 to June 2022, but this project will only look at 2018 forward. Looking at data from 2018 to the present represents two "normal" years before the pandemic in 2020, and two years after. The same 2018-2020 time period will be used for the other datasets.

Unexpectedly, the dataframe has 51 rows instead of 50, for 50 states. An examination of the RegionName column explains why:

In [4]:
print(home_state['RegionName'].unique())

['California' 'Texas' 'New York' 'Florida' 'Illinois' 'Pennsylvania'
 'Ohio' 'Michigan' 'Georgia' 'North Carolina' 'New Jersey' 'Virginia'
 'Washington' 'Massachusetts' 'Indiana' 'Arizona' 'Tennessee' 'Missouri'
 'Maryland' 'Wisconsin' 'Minnesota' 'Colorado' 'Alabama' 'South Carolina'
 'Louisiana' 'Kentucky' 'Oregon' 'Oklahoma' 'Connecticut' 'Iowa'
 'Mississippi' 'Arkansas' 'Kansas' 'Utah' 'Nevada' 'New Mexico'
 'West Virginia' 'Nebraska' 'Idaho' 'Hawaii' 'Maine' 'New Hampshire'
 'Rhode Island' 'Montana' 'Delaware' 'South Dakota' 'Alaska'
 'North Dakota' 'Vermont' 'District of Columbia' 'Wyoming']


On a quick glance, the dataset includes housing data for Washington DC, which explains the extra row. This will be kept in the dataset, and in the other datasets, should it exist.

Before looking further at the data, we will make the dataframe easier to work with by dropping all non-date columns, and turning the full state name into the row headers (the index). We will also limit our time frame to 2018 forward, as stated previously. Then we will check for missing (null) values, and values that do not make sense.

In [5]:
#Dropping non-date columns
home_state = home_state.drop(columns = ['RegionID','SizeRank','RegionType','StateName'])

#Setting the RegionName column to the index
home_state.set_index('RegionName',inplace=True)
home_state.columns = pd.to_datetime(home_state.columns)

#Using only 2018 data forward
home_state = home_state.loc[:,home_state.columns>='2018-01-31']
home_state.head(3)

,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
RegionName,,,,,,,,,,,,,,,,,,,,,
California,524332.0,529454.0,533849.0,537282.0,540032.0,542792.0,544534.0,545943.0,546844.0,548606.0,...,716941.0,721951.0,727440.0,734241.0,744446.0,756574.0,771569.0,786055.0,796986.0,799311.0
Texas,198850.0,199860.0,200905.0,201984.0,202990.0,203985.0,204837.0,205812.0,206665.0,207606.0,...,269194.0,273492.0,277149.0,281437.0,286128.0,291601.0,297048.0,303127.0,308619.0,313339.0
New York,289604.0,291009.0,292564.0,293987.0,295283.0,296518.0,297825.0,299224.0,300701.0,302030.0,...,370157.0,373228.0,376126.0,378994.0,382306.0,386106.0,390880.0,395849.0,400971.0,405105.0


In [6]:
#Check how many columns have null values
home_state.isna().any().sum()

9

In [7]:
#Checking the null values
home_state.loc[home_state.isna().any(axis=1),home_state.isna().any(axis=0)]

,2019-04-30,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2021-01-31,2021-02-28,2021-03-31,2021-12-31
RegionName,,,,,,,,,
Pennsylvania,195918.0,NaN,210186.0,NaN,215432.0,220976.0,223442.0,225964.0,250716.0
North Carolina,204405.0,219778.0,221591.0,223664.0,225924.0,NaN,235114.0,238807.0,284415.0
Connecticut,261170.0,NaN,NaN,279407.0,284038.0,291728.0,296108.0,300672.0,335400.0
Iowa,153319.0,158860.0,159318.0,159784.0,160709.0,NaN,NaN,166720.0,181196.0
Arkansas,129584.0,136683.0,137461.0,138381.0,139561.0,NaN,143998.0,145955.0,164183.0
Kansas,154429.0,163339.0,164503.0,166023.0,NaN,172243.0,174599.0,176985.0,195034.0
West Virginia,109469.0,113131.0,113511.0,114015.0,115024.0,116908.0,117806.0,118840.0,NaN
Idaho,276150.0,312878.0,317462.0,323821.0,331740.0,NaN,365133.0,377972.0,454341.0
Montana,280975.0,301076.0,303257.0,306344.0,310140.0,NaN,328053.0,334864.0,405374.0


There are few missing values in the dataframe, and it is best practice to avoid deleting needed data if possible. Interpolation can be used to fill in any missing values.

In [8]:
#Using interpolation, then rechecking number of columns with nulls
home_state.interpolate(method='linear',axis=1,limit=3,inplace=True)
home_state.isna().any().sum()

0

Next, let's check for data that does not make sense. Since this is a dataset of home values nationally, a wide range can be expected. However, since these are typical mid-tier home values, extreme values, from anywhere near 0 or less or values of several million dollars, are unlikely.

In [9]:
home_state.describe()

,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,...,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,242325.078431,243702.000000,244975.294118,246214.098039,247307.882353,248403.725490,249440.686275,250395.764706,251325.882353,252231.823529,...,324407.882353,328150.274510,331587.196078,335774.656863,340334.333333,345566.274510,350421.372549,355393.176471,359671.333333,363108.431373
std,113356.385431,114242.134691,114797.677621,115253.543475,115522.940516,116074.981297,116565.477150,116997.518664,117347.082817,117709.960621,...,147746.143057,149434.266262,151002.938959,152368.185319,154152.284490,156468.374484,159340.904579,162033.195234,164158.139863,164954.404978
min,105717.000000,106062.000000,106363.000000,106599.000000,106729.000000,106792.000000,106807.000000,106802.000000,107035.000000,107248.000000,...,124295.000000,125232.000000,126147.000000,128481.500000,130816.000000,133471.000000,134494.000000,135729.000000,136566.000000,137379.000000
25%,168113.000000,168932.500000,169412.500000,169779.500000,170105.000000,170635.000000,171064.000000,171588.500000,172026.000000,172522.000000,...,217616.500000,219444.000000,221045.500000,223614.500000,226550.500000,229608.000000,231950.000000,234295.500000,236398.000000,238277.000000
50%,228572.000000,229729.000000,230996.000000,232515.000000,232790.000000,232374.000000,232201.000000,233357.000000,234480.000000,235330.000000,...,288276.000000,290835.000000,293240.000000,298839.000000,304438.000000,309865.000000,311242.000000,315132.000000,318170.000000,321497.000000
75%,271524.000000,273603.500000,275345.500000,277656.500000,279799.000000,282489.500000,284738.500000,287281.000000,289025.000000,290354.500000,...,385556.500000,392619.500000,398514.000000,405315.000000,410752.500000,416998.500000,423494.000000,431222.500000,436097.500000,441821.000000
max,641874.000000,644403.000000,647410.000000,648966.000000,650269.000000,651714.000000,654047.000000,656736.000000,658223.000000,659123.000000,...,784049.000000,798344.000000,811929.000000,822449.000000,833782.000000,847135.000000,862489.000000,876786.000000,890452.000000,901942.000000


In [10]:
#Print minimum home value
print(home_state.describe().loc['min',:].min())

#Print maximum home value
print(home_state.describe().loc['max',:].max())

105717.0
901942.0


The minimum home value in this dataset is close to 100,000 and the maximum is around 900,000. Those values are within an expected range.

Now that the dataset is clean, an examination of 1) if there was an increase in the typical mid-tier home value, and 2) if so, how much, by state:

In [11]:
home_state_new = home_state.copy()

#Keep time period of 2018 forward
home_state.columns = pd.to_datetime(home_state.columns)
home_state_new = home_state_new.loc[:,~((home_state_new.columns>'2018-01-31') & (home_state_new.columns <'2022-06-30'))]

#Calculate price change and percent increase
home_state_new['Price Change'] = home_state_new.iloc[:,1]-home_state_new.iloc[:,0]
home_state_new['Overall_%_increase'] = round(((home_state_new.iloc[:,1]-home_state_new.iloc[:,0])/home_state_new.iloc[:,0])*100,2)
home_state_new.sort_values(by='Overall_%_increase',ascending=False,inplace=True)
home_state_new 

,2018-01-31 00:00:00,2022-06-30 00:00:00,Price Change,Overall_%_increase
RegionName,,,,
Idaho,235535.0,475113.0,239578.0,101.72
Arizona,236882.0,450629.0,213747.0,90.23
Utah,312983.0,577112.0,264129.0,84.39
Florida,228572.0,397280.0,168708.0,73.81
Nevada,270341.0,468427.0,198086.0,73.27
Montana,260898.0,448875.0,187977.0,72.05
Georgia,185582.0,316705.0,131123.0,70.66
Tennessee,175874.0,299571.0,123697.0,70.33
North Carolina,189048.0,320291.0,131243.0,69.42


As expected, the typical home value increased in all states between January 2018 and June 2022. However, the amount of the increase is significant, with the top eight states having an overall increase of over 70%. Even states with the smallest increases still saw a 20% increase in home values.

Let's further break this down into a year by year percent increase, to see when the largest home value increases occured. Were they before or after the start of the pandemic?

In [12]:
#Percent increase in home value from year to year
home_state_yr = home_state.transpose()
home_state_yr = home_state_yr.groupby(home_state_yr.index.year,axis=0).mean()
home_state_yr = round(home_state_yr.pct_change()*100,2)
home_state_yr = home_state_yr.transpose()
home_state_yr

,2018,2019,2020,2021,2022
RegionName,,,,,
California,NaN,1.86,5.26,17.09,14.22
Texas,NaN,4.79,3.93,15.26,16.98
New York,NaN,4.16,3.40,11.91,9.86
Florida,NaN,4.73,4.83,17.09,22.99
Illinois,NaN,2.04,2.14,11.06,10.06
Pennsylvania,NaN,4.32,5.11,14.03,9.79
Ohio,NaN,5.95,6.42,14.49,10.97
Michigan,NaN,6.24,5.76,14.72,11.03
Georgia,NaN,6.62,5.50,17.06,19.59


In [13]:
#Average of all percent increases per year
home_state_yr.mean()

2018          NaN
2019     4.426471
2020     4.930000
2021    14.442745
2022    12.898431
dtype: float64

As seen above, there were small percent increases in the typical mid-tier home value from 2018-2020, with the overall average increase per year coming in at under 5%. Then from 2020-2022, there were dramatic increases for each state, in most cases double digits. This corresponds to an average percent increase of over 14% in 2021, which seems to be dropping in 2022. To further quantify this, how many states saw a double digit percent increase in home values each year?

In [14]:
print('2018-2019: ',home_state_yr.loc[home_state_yr.loc[:,2019] >= 10,:].shape[0])
print('2019-2020: ',home_state_yr.loc[home_state_yr.loc[:,2020] >= 10,:].shape[0])
print('2020-2021: ',home_state_yr.loc[home_state_yr.loc[:,2021] >= 10,:].shape[0])
print('2021-2022: ',home_state_yr.loc[home_state_yr.loc[:,2022] >= 10,:].shape[0])

2018-2019:  1
2019-2020:  1
2020-2021:  43
2021-2022:  38


There is a stark difference between the number of states with double digit increases in home values pre- and post-start of the pandemic. The biggest increase came from 2020-2021, when 43 states saw double digit percent rise in their home values. In 2021-2022, increases occurred in 38 states, slightly down. In contrast, before 2020 this was only 1 state per year; that state was Idaho.

It is likely the overall home values per state are much lower than those in the large metro areas in each state, where a high concentration of Americans live. Let's find the largest metro area in each state, and use that data to determine how the home values have increased in these areas.

In [15]:
#Get info about the metro home values file
home_metro.info()
home_metro.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908 entries, 0 to 907
Columns: 275 entries, RegionID to 2022-06-30
dtypes: float64(270), int64(2), object(3)
memory usage: 1.9+ MB


,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
0,102001,0,United States,Country,NaN,128454.0,128801.0,129167.0,129911.0,130662.0,...,311669.0,315805.0,319728.0,324103.0,328848.0,334078.0,339362.0,344931.0,349975.0,354165.0
1,394913,1,"New York, NY",Msa,NY,225516.0,226863.0,228076.0,230462.0,232684.0,...,567599.0,572173.0,575746.0,579000.0,582977.0,587634.0,594089.0,601248.0,608793.0,614826.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,231193.0,231998.0,233231.0,235575.0,238089.0,...,846547.0,851306.0,858511.0,867810.0,878593.0,890931.0,908424.0,927686.0,944296.0,945642.0
3,394463,3,"Chicago, IL",Msa,IL,171271.0,171675.0,172198.0,173245.0,174354.0,...,285566.0,288245.0,290960.0,293700.0,296355.0,298940.0,302259.0,305752.0,309745.0,312752.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,131555.0,131660.0,131748.0,131961.0,132186.0,...,329869.0,336001.0,341515.0,347444.0,354921.0,363495.0,371655.0,381221.0,389992.0,397605.0


In the first row is the data for the entirety of the US. Let's remove this, and check the rest of the RegionType column for any other variations.

In [16]:
#Dropping the first row to eliminate the national data
home_metro = home_metro.iloc[1:]

#Checking the RegionType column
home_metro.RegionType.value_counts()

Msa    907
Name: RegionType, dtype: int64

The RegionType column shows the only data left is that of metro areas, but there are 907 of these. For the purposes of this project, only the largest metro in each state will be used. First, this dataframe will be formatted like the state-wide data, by turning the metro and state into the index, and dropping any non-date columns. Then we will check for any missing values before diving into the analysis.

In [17]:
#Remove state abbreviation from RegionName column
home_metro.loc[:,'RegionName'] = home_metro.loc[:,'RegionName'].str.split(',').str[0]

#Group by StateName, take largest metro area in each
home_metro.sort_values('SizeRank',inplace=True)
home_metro = home_metro.groupby(by='StateName',as_index=False).apply(lambda x:x.head(1))
home_metro.drop(columns=['SizeRank','RegionID','RegionType'],inplace=True)
home_metro.head(3)

,,RegionName,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,2000-07-31,2000-08-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
0,133,Anchorage,AK,146009.0,146216.0,146383.0,146524.0,146797.0,146905.0,147338.0,147332.0,...,326698.0,328546.0,326399.0,325473.0,324872.0,329512.0,332797.0,340538.0,347338.0,356732.0
1,49,Birmingham,AL,121404.0,121840.0,121990.0,122203.0,122785.0,123515.0,124301.0,124285.0,...,218215.0,220738.0,223685.0,227239.0,230146.0,233254.0,235885.0,239441.0,242271.0,244871.0
2,76,Little Rock,AR,99370.0,99539.0,99738.0,100170.0,100548.0,100822.0,101011.0,101252.0,...,176904.0,178642.0,181356.0,183713.0,186552.0,188429.0,189867.0,191568.0,193482.0,195775.0


In [18]:
#Set location information as index
home_metro = home_metro.set_index(['RegionName','StateName'])

#Dates from 2018 forward
home_metro = home_metro.loc[:,home_metro.columns>='2018-01-31']
home_metro.head(3)

,,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
RegionName,StateName,,,,,,,,,,,,,,,,,,,,,
Anchorage,AK,301388.0,301304.0,301109.0,300846.0,300899.0,301315.0,302334.0,302853.0,302715.0,302286.0,...,326698.0,328546.0,326399.0,325473.0,324872.0,329512.0,332797.0,340538.0,347338.0,356732.0
Birmingham,AL,160026.0,160862.0,161223.0,162000.0,162910.0,163631.0,164555.0,165333.0,166489.0,167198.0,...,218215.0,220738.0,223685.0,227239.0,230146.0,233254.0,235885.0,239441.0,242271.0,244871.0
Little Rock,AR,145963.0,146118.0,146271.0,146615.0,146900.0,147164.0,147556.0,147838.0,148229.0,148521.0,...,176904.0,178642.0,181356.0,183713.0,186552.0,188429.0,189867.0,191568.0,193482.0,195775.0


In [19]:
#Check for missing values
home_metro.isna().any().sum()

4

In [20]:
#Interpolate missing values then check again for nulls
home_metro = home_metro.interpolate(method='linear', axis=1,limit = 3)
home_metro.isna().any().sum()

0

In [21]:
#Overall percent increase in home values
home_metro_new = home_metro.loc[:,['2018-01-31','2022-06-30']]
home_metro_new['Price Change']= home_metro_new.iloc[:,1]-home_metro_new.iloc[:,0]
home_metro_new['Overall % increase'] = round(100*(home_metro_new.iloc[:,1]-home_metro_new.iloc[:,0])/home_metro_new.iloc[:,0],2)
home_metro_new = home_metro_new.sort_values('Overall % increase',ascending = False)
home_metro_new

,,2018-01-31,2022-06-30,Price Change,Overall % increase
RegionName,StateName,,,,
Boise City,ID,250890.0,517534.0,266644.0,106.28
Phoenix,AZ,249637.0,482463.0,232826.0,93.27
Salt Lake City,UT,335789.0,613471.0,277682.0,82.70
Charlotte,NC,214549.0,386038.0,171489.0,79.93
Atlanta,GA,216587.0,381361.0,164774.0,76.08
Las Vegas,NV,259258.0,453682.0,194424.0,74.99
Nashville,TN,262788.0,452102.0,189314.0,72.04
Dallas-Fort Worth,TX,237518.0,397605.0,160087.0,67.40
Indianapolis,IN,165570.0,270516.0,104946.0,63.38


Even the smallest increase in the typical mid-tier home value was still over 18%, applying to the Anchorage, AK market. That translates to an increase of approximately $55,000 over the 4.5 year period.

In contrast, the highest increase was the Boise, Idaho market, which saw home values double. A home buyer looking to save the full 20% for a downpayment would have needed around 50,000 dollars in 2018, but waiting a few years would see that amount go up to over 100,000. 

As an aside, we've looked at the home value increases, but which metro areas have the overall highest and the lowest home values as of mid-year 2022? 

In [22]:
#Top 5 highest home values
home_metro_new.sort_values('2022-06-30',ascending=False).head()

,,2018-01-31,2022-06-30,Price Change,Overall % increase
RegionName,StateName,,,,
Los Angeles-Long Beach-Anaheim,CA,640354.0,945642.0,305288.0,47.67
Urban Honolulu,HI,711522.0,937104.0,225582.0,31.70
Seattle,WA,487015.0,793263.0,306248.0,62.88
Boston,MA,468795.0,663494.0,194699.0,41.53
Denver,CO,411616.0,646474.0,234858.0,57.06


In [23]:
#Lowest 5 home values
home_metro_new.sort_values('2022-06-30',ascending=False).tail()

,,2018-01-31,2022-06-30,Price Change,Overall % increase
RegionName,StateName,,,,
Oklahoma City,OK,151304.0,216826.0,65522.0,43.30
Little Rock,AR,145963.0,195775.0,49812.0,34.13
Wichita,KS,133595.0,190677.0,57082.0,42.73
Jackson,MS,133186.0,186318.0,53132.0,39.89
Huntington,WV,98628.0,126746.0,28118.0,28.51


There are no surprises in either the five metro areas with the highest mid-tier home values, nor the five with the lowest. The highest is the LA/Long Beach/Anaheim metro area, with home values approaching $1 million, and the other areas are also known for having higher costs of living as well. The bottom five are metro areas of Southern states, where the cost of living is traditionally lower. 

With an overview of how home values have changed over time, let's examine two more pieces of data: home inventory levels and how fast homes are selling. This will paint a more complete picture of how else the housing market has been affected during the pandemic. We'll start with housing inventory.

In [24]:
#Examine the housing inventory dataframe
inventory_metro.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
0,102001,0,United States,Country,NaN,1424597.0,1381971.0,1438966.0,1525925.0,1627554.0,...,1135210.0,1126131.0,1062440.0,948819.0,823835.0,725043.0,715583.0,739448.0,807373.0,879047.0
1,394913,1,"New York, NY",Msa,NY,68792.0,68338.0,72824.0,79740.0,85755.0,...,63578.0,61886.0,58322.0,52225.0,44757.0,39560.0,39507.0,41517.0,44808.0,47371.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,20002.0,19836.0,21790.0,23704.0,25665.0,...,21430.0,20457.0,18589.0,15824.0,13528.0,12563.0,13620.0,14831.0,16363.0,17894.0


In [25]:
#Format the inventory_metro dataframe similarly to the home_metro dataframe

#Creating a function to apply to the other dataframes
def format_df(df):
    df = df.iloc[1:]
    df = df.drop(['RegionID','RegionType'],axis=1)
    df.loc[:,'RegionName'] = df.loc[:,'RegionName'].str.split(',').str[0]
    df = df.sort_values('SizeRank')
    df = df.groupby(by='StateName',as_index=False).apply(lambda x:x.head(1))
    df = df.drop(['SizeRank'],axis=1)
    df = df.set_index(['RegionName','StateName'])
    df.columns = pd.to_datetime(df.columns)
    return df

#Applying function to inventory dataset
inventory_metro = format_df(inventory_metro)

#Dates from 2018 forward
inventory_metro = inventory_metro.loc[:,inventory_metro.columns>='2018-01-31']

inventory_metro.head(3)

,,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
RegionName,StateName,,,,,,,,,,,,,,,,,,,,,
Anchorage,AK,1836.0,1777.0,1896.0,2044.0,2240.0,2415.0,2577.0,2662.0,2605.0,2481.0,...,1695.0,1590.0,1352.0,1091.0,849.0,737.0,739.0,850.0,995.0,1167.0
Birmingham,AL,6250.0,6176.0,6423.0,6732.0,7009.0,7235.0,7295.0,7308.0,7181.0,7042.0,...,4222.0,4166.0,3933.0,3620.0,3283.0,3015.0,2984.0,3010.0,3131.0,3292.0
Little Rock,AR,4934.0,4794.0,4947.0,5195.0,5553.0,5806.0,5917.0,5977.0,5945.0,5889.0,...,3304.0,3288.0,3127.0,2856.0,2547.0,2254.0,2196.0,2228.0,2375.0,2474.0


In [26]:
#Check for missing values
inventory_metro.isna().any().sum()

0

In [27]:
#Get inventory change over time
inventory_metro_new = inventory_metro.loc[:,['2018-01-31','2022-06-30']]
inventory_metro_new['Inventory Change']= inventory_metro_new.iloc[:,1]-inventory_metro_new.iloc[:,0]
inventory_metro_new['Overall % increase'] = round(100*(inventory_metro_new.iloc[:,1]-inventory_metro_new.iloc[:,0])/inventory_metro_new.iloc[:,0],2)
inventory_metro_new = inventory_metro_new.sort_values('Overall % increase',ascending = False)
inventory_metro_new

,,2018-01-31 00:00:00,2022-06-30 00:00:00,Inventory Change,Overall % increase
RegionName,StateName,,,,
Seattle,WA,7696.0,9943.0,2247.0,29.20
Minneapolis-St Paul,MN,10307.0,10583.0,276.0,2.68
Sioux Falls,SD,1066.0,969.0,-97.0,-9.10
Los Angeles-Long Beach-Anaheim,CA,20002.0,17894.0,-2108.0,-10.54
Portland,OR,7657.0,6842.0,-815.0,-10.64
Salt Lake City,UT,2586.0,2310.0,-276.0,-10.67
Boston,MA,10400.0,8947.0,-1453.0,-13.97
Detroit,MI,15149.0,12620.0,-2529.0,-16.69
Denver,CO,10508.0,8709.0,-1799.0,-17.12


We see a net decrease in available houses on the market in every large metro area, except Seattle, Washington, and the Minneapolis-St. Paul, Minnesota metro areas. However, even for these 2 metro areas, the increase in houses that are available is modest. Once again, let's do a year by year percent increase to further break this down.

In [28]:
#Group by year, then calculate percent increase
inventory_year = inventory_metro.groupby(inventory_metro.columns.year,axis=1).mean()
inventory_year_pct = (round(inventory_year.transpose().pct_change(),2)*100).transpose()
inventory_year_pct

,,2018,2019,2020,2021,2022
RegionName,StateName,,,,,
Anchorage,AK,NaN,-9.0,-18.0,-17.0,-36.0
Birmingham,AL,NaN,-7.0,-15.0,-25.0,-23.0
Little Rock,AR,NaN,-7.0,-14.0,-33.0,-21.0
Phoenix,AZ,NaN,0.0,-18.0,-16.0,-9.0
Los Angeles-Long Beach-Anaheim,CA,NaN,10.0,-19.0,-15.0,-26.0
Denver,CO,NaN,15.0,-16.0,-42.0,-22.0
Hartford,CT,NaN,-4.0,-22.0,-30.0,-38.0
Washington,DC,NaN,-10.0,-18.0,-4.0,-26.0
Salisbury,DE,NaN,-2.0,-21.0,-42.0,-28.0


At a glance, housing inventory for some metro areas increased and some decreased from 2018-2019, but after that there seems to be a decrease across the board. Quantifying how many metros had an inventory increase per year:

In [29]:
print('2018-2019: ',inventory_year_pct.loc[(inventory_year_pct[2019] > 0),:].index.nunique())
print('2019-2020: ',inventory_year_pct.loc[(inventory_year_pct[2020] > 0),:].index.nunique())
print('2020-2021: ',inventory_year_pct.loc[(inventory_year_pct[2021] > 0),:].index.nunique())
print('2021-2022: ',inventory_year_pct.loc[(inventory_year_pct[2022] > 0),:].index.nunique())

2018-2019:  29
2019-2020:  0
2020-2021:  0
2021-2022:  0


There were 29 states where the number of houses available increased from 2018-2019. After this time period, all metro areas had a decrease for the next several years.

The last piece of the puzzle we will investigate is the time it takes for houses to go under contrat (in "pending" status on Zillow).

In [30]:
pending_metro.head(3)

,RegionID,SizeRank,RegionName,RegionType,StateName,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
0,102001,0,United States,Country,NaN,30.0,22.0,20.0,20.0,22.0,...,10.0,11.0,11.0,13.0,13.0,11.0,9.0,7.0,7.0,7.0
1,394913,1,"New York, NY",Msa,NY,46.0,36.0,35.0,36.0,39.0,...,30.0,31.0,34.0,36.0,42.0,38.0,31.0,22.0,20.0,21.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,17.0,15.0,16.0,16.0,18.0,...,13.0,13.0,13.0,13.0,13.0,12.0,11.0,10.0,11.0,12.0


In [31]:
#Format the dataframe using the previously defined function
pending_metro = format_df(pending_metro)

#Dates from 2018 forward, dataset starts in March
pending_metro = pending_metro.loc[:,pending_metro.columns>='2018-03-31']

pending_metro.head(3)

,,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30
RegionName,StateName,,,,,,,,,,,,,,,,,,,,,
Anchorage,AK,36.0,23.0,22.0,23.0,25.0,30.0,36.0,45.0,52.0,62.0,...,9.0,12.0,14.0,14.0,11.0,8.0,5.0,4.0,4.0,5.0
Birmingham,AL,24.0,15.0,13.0,14.0,16.0,19.0,22.0,25.0,26.0,29.0,...,6.0,6.0,6.0,6.0,6.0,8.0,8.0,7.0,5.0,5.0
Little Rock,AR,50.0,43.0,38.0,40.0,43.0,45.0,45.0,46.0,51.0,59.0,...,12.0,14.0,15.0,18.0,18.0,15.0,10.0,8.0,7.0,7.0


In [32]:
#Check for missing values
pending_metro.isna().any().sum()

39

In [33]:
#Display all NaNs
pending_metro.loc[pending_metro.isna().any(axis=1),pending_metro.isna().any(axis=0)]

,,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,...,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31
RegionName,StateName,,,,,,,,,,,,,,,,,,,,,
Fargo,ND,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Huntington,WV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.0,28.0,23.0,24.0,24.0,34.0,36.0,34.0,20.0,15.0


All missing values in this dataset belong to either Fargo, North Dakota or Huntington, West Virginia. Based on the amount of data missing for both metro areas, these can be dropped entirely.

In [34]:
#Drop Fargo and Huntington 
pending_metro = pending_metro.drop(['Huntington','Fargo'],level='RegionName', axis=0)

#Check again for nulls
pending_metro.isna().any().sum()

0

In [35]:
#Get change over time in number of days to pending
pending_metro_new = pending_metro.loc[:,['2018-03-31','2022-06-30']]
pending_metro_new['Days to Pending Change']= pending_metro_new.iloc[:,1]-pending_metro_new.iloc[:,0]
pending_metro_new['Overall % increase'] = round(100*(pending_metro_new.iloc[:,1]-pending_metro_new.iloc[:,0])/pending_metro_new.iloc[:,0],2)
pending_metro_new = pending_metro_new.sort_values('Overall % increase',ascending = False)
pending_metro_new

,,2018-03-31 00:00:00,2022-06-30 00:00:00,Days to Pending Change,Overall % increase
RegionName,StateName,,,,
Milwaukee,WI,18.0,19.0,1.0,5.56
Seattle,WA,7.0,6.0,-1.0,-14.29
Denver,CO,6.0,5.0,-1.0,-16.67
Los Angeles-Long Beach-Anaheim,CA,17.0,12.0,-5.0,-29.41
Salt Lake City,UT,11.0,6.0,-5.0,-45.45
Boise City,ID,15.0,8.0,-7.0,-46.67
Boston,MA,15.0,7.0,-8.0,-53.33
New York,NY,46.0,21.0,-25.0,-54.35
Portland,OR,15.0,6.0,-9.0,-60.00


Similar to previous analysis of the other datasets, there is a decrease in sale time for houses on the market for almost all states. The one exception is Milwaukee, Wisconsin; however, the number of days for this city only increased by 1 from 2018 to 2022. Assuming previous patterns hold, we would see a dramatic percent decrease in time on the market from 2019-2020 for most states, and then another (less dramatic) dip from 2021-2022. 

Let's check if this is true by calculating the overall average time on the market per year:

In [36]:
#Getting average number of days on market per year
pending_metro.groupby(pending_metro.columns.year, axis=1).mean().mean()

2018    25.677551
2019    28.217687
2020    20.603741
2021    10.685374
2022     8.690476
dtype: float64

As we suspected, the average number of days a house was on the market was increasing before the pandemic, after which it began to rapidly decline. Buyers searching pre-pandemic had on average over 25 days before a house went under contract, and now this has dropped to single digits.

Last, let's find the metro areas with the longest and the shortest turn around times. 

In [37]:
pending_metro.loc[:,'2022-06-30'].describe()

count    49.000000
mean      7.122449
std       3.643674
min       3.000000
25%       5.000000
50%       6.000000
75%       8.000000
max      21.000000
Name: 2022-06-30 00:00:00, dtype: float64

In [38]:
#Finding metro with shortest turnaround time based on describe above
pending_metro.loc[pending_metro.loc[:,'2022-06-30']==3,'2022-06-30']

RegionName  StateName
Cincinnati  OH           3.0
Name: 2022-06-30 00:00:00, dtype: float64

In [39]:
#Finding metro with longest turnaround time
pending_metro.loc[pending_metro.loc[:,'2022-06-30']==21,'2022-06-30']

RegionName  StateName
New York    NY           21.0
Name: 2022-06-30 00:00:00, dtype: float64

Cincinnati, Ohio is down to three days as of the end of June 2022. At this number of days, buyers would likely have to put down an offer almost as soon as they view a home to have any chance of acceptance. Meanwhile, New York City is at 21 days, a little under half of the time from before the pandemic. 

Another interesting facet of this data is that even at the upper 75th percentile of days on the market, this only amounts to eight days. This then leaves any metro areas with double digit days in this upper percentile.

Using data from Zillow, we have managed to paint a clear picture of how the housing market boomed during the pandemic. Before 2020, housing prices were increasing at a modest pace, homes were more available, and homes were generally on the market for several weeks before going under contract. During and after 2020 the housing situation changed drastically, leaving many families unable to buy. Housing prices shot up in the period from 2020-2021, in some markets even doubling, while the average time a home was on the market went down to typically under a week. Furthermore, housing inventory has declined considerably. While price increases have slowed and inventory has picked up, the market has not yet stabilized and housing metrics continue to move in an undesirable direction.

If we were to expand this project further, a signficant area of interest would be the rental market, the data for which is also available on Zillow. Renting is increasing across the country, and a future project would want to quantify this increase exactly, and determine which markets have been the most affected. These datasets could be tied into wage data available via the U.S. Census. This would allow a better understand how affordable housing is given the average family's income, per state or metro area.